In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import pyopencl as cl
import pyopencl.array
import pyopencl.clrandom
import loopy as lp

In [ ]:
tol = 1e-12
mnorm = lambda x: np.max(np.abs(x))

In [ ]:
ctx = cl.create_some_context()
queue = cl.CommandQueue(ctx)

In [ ]:
n = 16*16
x_d = cl.clrandom.rand(queue, n, dtype=np.double)
A_d = cl.clrandom.rand(queue, (n, n), dtype=np.double)

x = x_d.get()
A = A_d.get()
b = A.dot(x)

In [ ]:
knl = lp.make_kernel(
        "{ [i,j]: 0<=i,j<n }",
        "out[i] = sum(j, A[i,j]*x[j])", assumptions="n>=0")
knl = lp.set_options(knl, "write_cl")

#knl = lp.split_iname(knl, "i", 128, outer_tag="g.0", inner_tag="l.0")



In [ ]:
# knl = lp.split_iname(knl, "i", 128, 
#                      outer_tag="g.0",
#                     inner_tag="l.0")

In [ ]:
evt, (b_d,) = knl(queue, A=A_d, x=x_d)
evt.wait()

mnorm(b_d.get()-b)<tol

In [ ]:
print knl